In [1]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
np.set_printoptions(threshold=np.nan)
import json
import string

/home/ps/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
"""Set False for testing"""
training=False

In [3]:
"""Loading max_frames, speaker_list and count of files"""
with open("dataset/max_frames_speakers_count.pkl") as f:
    max_frames, speakers, count = pickle.load(f)
    print max_frames
    print len(speakers)
    print count

500
108
1


In [4]:
"""Set of Hyperparameters along with the phoneme list"""
batch_size = 25
num_features = 13
speak_feats = 16 #features in speaker embedding
num_speakers = len(speakers)
num_conv_filters = 64
num_conv_layers = 4 #layers in convolutional network excluding zeroth layer
num_rnn_layers = 3
dropout = 0.85
num_hidden_gru = 1024
beam_width= 20

#Phonemes
silence = "SIL"

phonemes = [silence,'AA','AE','AH','AO','AW','AY','B','CH','D','DH','EH',
            'ER','EY','F','G','HH','IH','IY','JH','K','L','M','N','NG','OW','OY',
            'P','R','S','SH','T','TH','UH','UW','V','W','Y','Z','ZH']
phoneme_dict = dict()
for i in range(len(phonemes)):
    phoneme_dict[phonemes[i]] = i


num_phonemes = len(phonemes)
num_labels = num_phonemes**2
ctc_classes = num_labels + 1 

In [5]:
x = tf.placeholder(tf.float32, [batch_size, max_frames,num_features]) ##MFCC features
y = tf.sparse_placeholder(tf.int32) ##Phoneme pair labels
y_length_inp = tf.placeholder(tf.int32, [batch_size])
y_length_pred = tf.placeholder(tf.int32, [batch_size])
speaker_selector = tf.placeholder(tf.float32, [batch_size, num_speakers]) #One hot speaker selector matrix

In [6]:
"""16 feature speaker embedding for better distinction in speakers"""
with tf.variable_scope("speaker_embedding"):
    speaker = tf.get_variable("speaker",
                              shape=(num_speakers,speak_feats),
                              dtype= tf.float32,
                              initializer=tf.contrib.layers.xavier_initializer(),
                              trainable=True)
    speaker_matrix = tf.matmul(speaker_selector, speaker)
speaker_matrix

<tf.Tensor 'speaker_embedding/MatMul:0' shape=(25, 16) dtype=float32>

In [7]:
"""fully connected layer that takes speaker as the input and sends state to the convolutional layers"""
with tf.variable_scope("conv_speaker"):
    conv_speaker_outputs = num_conv_filters
    conv_speaker_fc = tf.contrib.layers.fully_connected(speaker_matrix,
                                                        conv_speaker_outputs,
                                                        activation_fn= tf.nn.softsign)
conv_speaker_fc

<tf.Tensor 'conv_speaker/fully_connected/Softsign:0' shape=(25, 64) dtype=float32>

In [8]:
"""fully connected layer that takes speaker as the input and sends state to the Bidirectional Gru layers"""
with tf.variable_scope("rnn_speaker"):
    rnn_speaker_outputs = num_hidden_gru
    rnn_speaker = tf.contrib.layers.fully_connected(speaker_matrix,
                                                    rnn_speaker_outputs,
                                                    activation_fn= tf.nn.softsign)
rnn_speaker
    

<tf.Tensor 'rnn_speaker/fully_connected/Softsign:0' shape=(25, 1024) dtype=float32>

In [9]:
"""Preprocessing of speaker_conv for the convolution layers"""
with tf.variable_scope("speaker_state_conv"):
    #finding speaker state for cnn
    length = max_frames
    breadth = num_features
    height = num_conv_filters
    depth = batch_size
    multiply = length*breadth

    ##transforming speaker embedding for convolutional layers
    a = tf.expand_dims(conv_speaker_fc , axis = 0)
    b = tf.contrib.seq2seq.tile_batch(a, multiplier=multiply)
    c = tf.reshape(b,[length,breadth,batch_size,-1]) 
    speaker_state_conv = tf.transpose(c,[2,0,1,3])
    
speaker_state_conv

<tf.Tensor 'speaker_state_conv/transpose:0' shape=(25, 500, 13, 64) dtype=float32>

In [10]:
"""zeroth convolutional layer"""
with tf.variable_scope("zero_conv_layer"):
    conv0 =  tf.layers.conv2d(tf.expand_dims(x, axis = 3),
                              filters = num_conv_filters,
                              kernel_size = (9,5),
                              padding = "SAME")
    
network = tf.expand_dims(conv0 , 0)

In [11]:
"""Stacking convolutional layers with batch normalization and speaker embedding and relu6"""
with tf.variable_scope("conv_layers"):
    for i in range(num_conv_layers):
        conv = tf.layers.conv2d(network[i],
                                filters = num_conv_filters,
                                kernel_size = (9,5),
                                padding = "SAME")

        conv = tf.contrib.layers.batch_norm(conv)


        #multipying speaker embedding with conv+bn output
        conv_layer = tf.multiply(speaker_state_conv, conv)
        conv_layer = tf.add(conv_layer, network[i])

        #applying relu6
        conv_layer = tf.nn.relu6(conv_layer)
        
        #adding to networks
        network = tf.concat([network,tf.expand_dims(conv_layer,0)],0)

In [12]:
"""setting up dropout layer after convolutions"""
with tf.variable_scope("recurrent_input"):
    conv_output = tf.nn.dropout(network[num_conv_layers], keep_prob = dropout)
    rnn_input = tf.reshape(conv_output,[batch_size,max_frames,-1])
    
rnn_input

<tf.Tensor 'recurrent_input/Reshape:0' shape=(25, 500, 832) dtype=float32>

In [13]:
def gru_cell():
    return tf.nn.rnn_cell.GRUCell(num_hidden_gru)

In [14]:
"""Making the correct input tensor for the rnn """
with tf.variable_scope("rnn_preprocess"):
    inputs = rnn_input
    rnn_input_embed = tf.contrib.seq2seq.tile_batch(tf.expand_dims(rnn_speaker,0), multiplier = max_frames)
    rnn_input_embed = tf.transpose(rnn_input_embed,[1,0,2])
    inputs = tf.concat([inputs, rnn_input_embed], axis = 2)
inputs

<tf.Tensor 'rnn_preprocess/concat:0' shape=(25, 500, 1856) dtype=float32>

In [15]:
"""Creating the bi directional gru layers"""
for i in range(num_rnn_layers):
    with tf.variable_scope("RNN_BI"+str(i+1)):
        outputs, output_states = tf.nn.bidirectional_dynamic_rnn(gru_cell(),
                                                                 gru_cell(),
                                                                 inputs,
                                                                 initial_state_fw = rnn_speaker,
                                                                 initial_state_bw = rnn_speaker)

        inputs = tf.concat(outputs, axis = 2)

rnn_output = inputs
rnn_output

<tf.Tensor 'RNN_BI3/concat:0' shape=(25, 500, 2048) dtype=float32>

In [16]:
"""Fully connected layer to convert rnn output to ctc input"""
with tf.variable_scope("rnn_postprocess"):
    rnn_output = tf.nn.dropout(rnn_output, keep_prob = dropout)
 
    ctc_input = tf.contrib.layers.fully_connected(rnn_output,
                                                  ctc_classes,
                                                  activation_fn= tf.nn.softmax)
ctc_input

<tf.Tensor 'rnn_postprocess/fully_connected/Reshape_1:0' shape=(25, 500, 1601) dtype=float32>

In [17]:
"""Calculating CTC loss over the batch"""
with tf.variable_scope("CTC_loss"):
    y_true = tf.sparse_tensor_to_dense(y)
    ctc_loss = tf.keras.backend.ctc_batch_cost(y_true,
                                               ctc_input,
                                               input_length = tf.reshape(y_length_pred,[-1,1]),
                                               label_length = tf.reshape(y_length_inp,[-1,1]))
ctc_loss

<tf.Tensor 'CTC_loss/ExpandDims:0' shape=(25, 1) dtype=float32>

In [18]:
"""A Custom Built Beam Search For CTC"""
with tf.variable_scope("beam_search"):


    batch_const = tf.constant([[j for j in range(batch_size)]])
    batch_const = tf.transpose(tf.contrib.seq2seq.tile_batch(batch_const, multiplier=beam_width),[1,0])
    
    length_filter = tf.tile(tf.expand_dims(y_length_pred,axis=1),[1,max_frames])
    temp_mask = tf.constant([[j for j in range(max_frames)]])
    temp_mask = tf.tile(temp_mask,[batch_size,1])
    frames_mask = tf.less(temp_mask,length_filter)
    frames_mask = tf.tile(tf.expand_dims(frames_mask,axis=2),[1,1,ctc_classes-1])
    blank_mask = tf.expand_dims(tf.ones([batch_size,max_frames],dtype=tf.float32),axis=2)
    blank_mask = tf.cast(blank_mask,tf.bool)
    frames_mask = tf.concat([frames_mask,blank_mask],axis=2)
    print frames_mask
    frames_mask = tf.cast(frames_mask,tf.float32)
    ctc_input_masked = tf.multiply(ctc_input,frames_mask)
    print ctc_input_masked
    ctc_log = tf.log(ctc_input_masked)
    frames = tf.unstack(ctc_log,axis=1)
    
    topk = tf.nn.top_k(frames[0],
                       k=beam_width,
                       sorted=False)
    print topk
    values = topk[0]
    indices = topk[1]

    values = tf.transpose(values, [1,0])
    indices = tf.transpose(indices, [1,0])

    values = tf.contrib.seq2seq.tile_batch(values, multiplier=beam_width)
    indices = tf.contrib.seq2seq.tile_batch(indices, multiplier=beam_width)
    indices = tf.expand_dims(indices,axis=0)
    print indices
    print values


    for i in range(1,len(frames)):
        topk_next = tf.nn.top_k(frames[i],k=beam_width,sorted=False)
        values_next = topk_next[0]
        indices_next = topk_next[1]
        values_next = tf.transpose(values_next, [1,0])
        indices_next = tf.transpose(indices_next, [1,0])
        
        values_next = tf.tile(values_next, tf.constant([beam_width,1],dtype=tf.int32))
        indices_next = tf.tile(indices_next, tf.constant([beam_width,1],dtype=tf.int32))
        
        indices = tf.concat([indices,tf.expand_dims(indices_next,axis=0)],axis=0)

        values = tf.add(values,values_next)
        values = tf.transpose(values,[1,0])
        tempk = tf.nn.top_k(values,k=beam_width,sorted=False)
        values = tf.contrib.seq2seq.tile_batch(tf.transpose(tempk[0],[1,0]), multiplier=beam_width)
        indices_new = tf.stack([batch_const,tempk[1]],axis=2)

        indices = tf.gather_nd(tf.transpose(indices,[2,1,0]), indices_new)
        indices = tf.transpose(indices, [1,0,2])
        indices = tf.contrib.seq2seq.tile_batch(indices, multiplier=beam_width)
        indices = tf.transpose(indices, [2,0,1])

    final_beam = tf.transpose(indices,[2,0,1])
    final_beam, _ = tf.split(final_beam,[1,-1],axis=2)
    final_path = tf.reshape(final_beam,[batch_size,-1])
    print final_path


Tensor("beam_search/concat:0", shape=(25, 500, 1601), dtype=bool)
Tensor("beam_search/Mul:0", shape=(25, 500, 1601), dtype=float32)
TopKV2(values=<tf.Tensor 'beam_search/TopKV2:0' shape=(25, 20) dtype=float32>, indices=<tf.Tensor 'beam_search/TopKV2:1' shape=(25, 20) dtype=int32>)
Tensor("beam_search/ExpandDims_3:0", shape=(1, 400, 25), dtype=int32)
Tensor("beam_search/tile_batch_1/Reshape:0", shape=(400, 25), dtype=float32)
Tensor("beam_search/Reshape:0", shape=(25, 500), dtype=int32)


In [19]:

def strip_punctuation(s):
        return ''.join(c for c in s if c not in '!"#$%&\()*+./<=>?@[\\]^_`{|}~')
    
with open('result.pkl') as f:  
    words_phonemes = pickle.load(f)


def get_phoneme_pairs(sentence,dict):
    """Takes a sentence and returns its phoneme pairs including SILENCE at start and end"""
    answer = [silence]
    sentence= strip_punctuation(sentence)
    words = sentence.replace(':',' ').replace(';',' ').replace(',',' ').split()
    
    for i in words:
        if i[-1] in ",;:":
            if i[:-1]!='':
                for j in dict[i[:-1].upper()]:
                    answer.append(j)
            answer.append(silence)
        else:
            for j in dict[i.upper()]:
                answer.append(j)
            answer.append(silence)
    answer[-1] = silence
    
    final_answer = []
    for i in range(len(answer)-1):
        final_answer.append((answer[i],answer[i+1]))
        
    return final_answer


In [20]:
def convert_to_labels(phoneme_pairs, label_dict):
    """Converts Phoneme Pairs to Labels"""
    labels = []
    for i in phoneme_pairs:
        labels.append(label_dict[i[0]]*(len(label_dict))+label_dict[i[1]])
    return labels


In [21]:
def labels_to_phoneme_pairs(labels, phoneme_list):
    """Converts labels to Phoneme Pairs"""
    phoneme_pairs = []
    for i in labels:
        if i ==-1 or i == (ctc_classes - 1):
            phoneme_pairs.append("-")
        else:
            x = i%(len(phoneme_list))
            y = i/(len(phoneme_list))
            phoneme_pairs.append((phonemes[y],phonemes[x]))
    return phoneme_pairs


In [22]:
"""Learning rate and annealing rate are set for the adam optimizer"""
global_step = tf.Variable(0, trainable=False)

starter_learning_rate = 2e-4

learning_rate = tf.train.exponential_decay(starter_learning_rate, 
                                           global_step,
                                           1000,
                                           0.95,
                                           staircase=True)

# Passing global_step to minimize() will increment it at each step.
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

minimize = optimizer.minimize(ctc_loss,global_step = global_step)

In [23]:
def get_sparse_parameters(labels_batch):
    """Returns parameters required for a sparse tensor for the phonemes"""
    indices=[]
    values=[]
    x = len(labels_batch)
    max_len = 0
    for i in range(len(labels_batch)):
        for j in range(len(labels_batch[i])):
            indices.append([i,j])
        values+=labels_batch[i]
        max_len=max(max_len,len(labels_batch[i]))
    
    return (indices,values,(x,max_len))    

In [24]:
def get_y_length_inp(labels_batch):
    """Returns the sequence lengths of input phoneme pairs"""
    y_len = []
    for i in range(len(labels_batch)):
        y_len.append(len(labels_batch[i]))
    return y_len

In [25]:
def get_speaker_one_hot(speaker_list, speakers):
    """Returns speaker one hot encoding for the list"""
    one_hot_speaker = np.zeros((len(speaker_list),len(speakers)))
    for i in range(len(speaker_list)):
        one_hot_speaker[i][speakers.index(speaker_list[i])]=1
    return one_hot_speaker

In [26]:
def pred_dur(ax,y_length_pred_test):
    """returns the final predicted phonemes along with durations in human readable form"""
    final_dur_list=[]
    for i in range(batch_size):
        out = ax[i]
        dur_list = list()
        out = np.insert(out,0,0)
        a = labels_to_phoneme_pairs(out[:y_length_pred_test[i]],phonemes)
        time_frame=1
        prev_pair = a[0]
        for phoneme_pairs_a in a:
            try:
                (b,c)=phoneme_pairs_a
                if len(dur_list)!=0 and phoneme_pairs_a==prev_pair:
                    time_frame+=1
                else:
                    dur_list.append([prev_pair[1],time_frame])
                    prev_pair = phoneme_pairs_a
                    time_frame=1
            except:
                time_frame+=1
        dur_list.append([prev_pair[1],time_frame])
        final_dur_list.append(dur_list[2:])
        
    return final_dur_list

In [27]:
def pred_pairs_dur_PPER(ax,y_length_pred_test):
    """returns the final predicted phonemes along with durations in human readable form for calculating PPER"""

    final_dur_list=[]
    for i in range(batch_size):
        out = ax[i]
        dur_list = list()
        out = np.insert(out,0,0)
        a = labels_to_phoneme_pairs(out[:y_length_pred_test[i]],phonemes)
        time_frame=1
        prev_pair = a[0]
        for phoneme_pairs_a in a:
            try:
                (b,c)=phoneme_pairs_a
                if len(dur_list)!=0 and phoneme_pairs_a==prev_pair:
                    time_frame+=1
                else:
                    dur_list.append(prev_pair)
                    prev_pair = phoneme_pairs_a
                    time_frame=1
            except:
                time_frame+=1
        dur_list.append(prev_pair)
        final_dur_list.append(dur_list[2:])
        
    return final_dur_list

In [28]:
def compare(true,pred):
    """Comparison algorithm for true and predicted phoneme pairs"""
    ptr1 = 0
    ptr2 = 0
    incorrect=0
    window_size=5
    total = 0
    for i in range(len(true)):
        if ptr1>len(true)-1 or ptr2>len(pred)-1:
            break
        if true[ptr1]==pred[ptr2]:
            ptr1+=1
            ptr2+=1
            total += 1
        else:
            try:
                jump = pred[ptr2:min(ptr2+window_size,len(pred))].index(true[ptr1]) 
                incorrect += jump
                total+=jump
                ptr2+=jump
            except:
                ptr1+=1
                incorrect += 1
                total+=1
    return total,incorrect

In [29]:
def PPER(test_data):
    """returns phoneme pair error rate"""
    no_of_batches=len(test_data)/batch_size
    ptr=0
    incorrect_pairs=0
    total_pairs=0
    for k in range(no_of_batches):        
        inp = test_data[ptr:ptr+batch_size]
        inp = zip(*inp)
        x_test = np.asarray(inp[2])
        temp = inp[1]
        y_labels = []
        for sentence in temp:
            y_labels.append(convert_to_labels(get_phoneme_pairs(sentence,words_phonemes),phoneme_dict))
        y_test = get_sparse_parameters(y_labels)
        y_length_inp_test = get_y_length_inp(y_labels)
        #print 'input', y_length_inp_train
        y_length_pred_test = list(inp[3])
        #print "pred", y_length_pred_train
        speaker_selector_test = get_speaker_one_hot(inp[0], speakers)
        ptr+=batch_size
        ax = sess.run(final_path,{x:x_test, y:y_test, y_length_inp:y_length_inp_test,y_length_pred:y_length_pred_test,
                                                 speaker_selector:speaker_selector_test})
        if not k:
            print labels_to_phoneme_pairs(ax[0][:200],phonemes)
        pair_dur = pred_pairs_dur_PPER(ax,y_length_pred_test)
        for i in range(batch_size):
            p_pairs = labels_to_phoneme_pairs(y_labels[i],phonemes)
            p_pairs_pred = pair_dur[i]
            total,incorrect = compare(p_pairs, p_pairs_pred)
            total_pairs+=total
            incorrect_pairs+=incorrect
    return (float(incorrect_pairs*100)/total_pairs)

In [30]:
"""Initialize session"""
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [31]:
"""For using previous models, restore, else comment"""
saver = tf.train.Saver()
saver.restore(sess, "saved_models/epoch10/model.ckpt")

INFO:tensorflow:Restoring parameters from saved_models/epoch10/model.ckpt


In [32]:
%%time
"""For Training only- set number of epochs. Calculates error% at every epoch"""
if training:
    epoch = 1
    cur_error=0
    min_error=100
    with open("dataset/1.pkl") as f:
                data = pickle.load(f)
    for i in range(epoch):
        for j in range(1):
    #         with open("dataset/"+str(j+1)+".pkl") as f:
    #             data = pickle.load(f)
            train_data, test_data, _, _ = train_test_split(data,data, test_size=0.1, random_state=42)
            no_of_batches=len(train_data)/batch_size
            ptr=0
            for k in range(no_of_batches):
                if k%100==0:
                    print k
                inp = train_data[ptr:ptr+batch_size]
                inp = zip(*inp)
                x_train = np.asarray(inp[2])
                temp = inp[1]
                y_labels = []
                for sentence in temp:
                    y_labels.append(convert_to_labels(get_phoneme_pairs(sentence,words_phonemes),phoneme_dict))
                y_train = get_sparse_parameters(y_labels)
                y_length_inp_train = get_y_length_inp(y_labels)
                y_length_pred_train = list(inp[3])
                speaker_selector_train = get_speaker_one_hot(inp[0], speakers)
                ptr+=batch_size

                sess.run(minimize,{x:x_train, y:y_train, y_length_inp:y_length_inp_train,y_length_pred:y_length_pred_train,
                                   speaker_selector:speaker_selector_train})
        print "Epoch - ",str(i)
        cur_error = PPER(test_data[:100])
        print "ERROR- ",str(cur_error)
        if cur_error < min_error:
            save_path = saver.save(sess,"saved_models/epoch"+str(i)+"/model.ckpt")
            print "Saved @ ", save_path
            min_error = cur_error


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs


In [ ]:
"""Predicts and stores durations to be used in duration prediction model"""
durations = []
speaker_dur = []
file_path = []
for j in range(1):
    with open("dataset/"+str(j+1)+".pkl") as f:
        data = pickle.load(f)
    train_data, test_data, _, _ = train_test_split(data,data, test_size=0.1, random_state=42)
    no_of_batches=len(train_data)/batch_size
    ptr=0
    for k in range(no_of_batches):
        print k
        inp = train_data[ptr:ptr+batch_size]
        inp = zip(*inp)
        x_train = np.asarray(inp[2])
        temp = inp[1]
        y_labels = []
        for sentence in temp:
            y_labels.append(convert_to_labels(get_phoneme_pairs(sentence,words_phonemes),phoneme_dict))
        y_train = get_sparse_parameters(y_labels)
        y_length_inp_train = get_y_length_inp(y_labels)
        #print 'input', y_length_inp_train
        y_length_pred_train = list(inp[3])
        #print "pred", y_length_pred_train
        speaker_selector_train = get_speaker_one_hot(inp[0], speakers)
        ptr+=batch_size
        durations += pred_dur(sess.run(final_path,{x:x_train, y:y_train, y_length_inp:y_length_inp_train,y_length_pred:y_length_pred_train,
                                     speaker_selector:speaker_selector_train}),y_length_pred_train)
        speaker_dur += list(inp[0])
        file_path += list(inp[4])
print len(durations)

In [ ]:
"""Pickles durations of phonemes of sentences"""
speaker_info = []
speaker_info.append(speaker_dur)
speaker_info.append(file_path)
speaker_info = zip(*speaker_info)
print speaker_info[:10]
for i in  range(len(speaker_info)):
    speaker_info[i] = list(speaker_info[i])
print speaker_info[:10]
with open("duration_pred.pkl",'w') as f:
    pickle.dump([durations,speaker_info],f)

In [33]:
with open("dataset/1.pkl") as f:
    data = pickle.load(f)
train_data, test_data, _, _ = train_test_split(data,data, test_size=0.1, random_state=42)

print PPER(train_data[:100])
print PPER(test_data[:100])

[('SIL', 'DH'), ('DH', 'AH'), ('AH', 'SIL'), ('AH', 'SIL'), '-', '-', '-', '-', '-', '-', ('SIL', 'M'), ('M', 'AE'), '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ('AE', 'N'), ('N', 'SIL'), '-', '-', '-', '-', '-', ('SIL', 'W'), ('W', 'AA'), ('W', 'AA'), '-', '-', ('AA', 'Z'), ('Z', 'SIL'), ('Z', 'SIL'), '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ('SIL', 'P'), ('SIL', 'P'), ('P', 'R'), ('R', 'AH'), '-', '-', '-', ('AH', 'N'), ('AH', 'N'), '-', '-', '-', '-', ('N', 'AW'), '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ('AW', 'N'), ('N', 'S'), '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ('S', 'T'), ('S', 'T'), ('T', 'SIL'), '-', '-', '-', ('SIL', 'D'), ('D', 'EH'), '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ('EH', 'D'), ('D', 'SIL'), '-', '-', '-', ('SIL', 'AA'), ('SIL', 'AA'), '-', '-', '-', '-', ('AA', 'N'), ('N', 'SIL'), ('N', 'SIL'), '-', '-', '-', '-', ('SIL', 'ER'), ('SIL', 'ER'), '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ('ER', 'AY